In [3]:
import pandas as pd
import numpy as np
import os

In [13]:
game_files = [
    'AuthorizedAttendanceReportCsv_Round 2_ Hapoel Tel Aviv 🚗.csv',
    'AuthorizedAttendanceReportCsv_Round 4_ Hapoel Holon 🏠.csv',
    'AuthorizedAttendanceReportCsv_Round 8_ Hapoel Afula 🏠.csv',
    'AuthorizedAttendanceReportCsv_🏠 מחזור 9_ הפועל ״בנק יהב״ ירושלים -הפועל גליל עליון.csv',
    'AuthorizedAttendanceReportCsv_🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת"א.csv',
    'AuthorizedAttendanceReportCsv_ליגת ווינר סל מחזור 17_ גלבוע גליל 🏠.csv',
    'AuthorizedAttendanceReportCsv_ליגת ווינר סל מחזור 18_ הרצליה 🏠 .csv',
    'AuthorizedAttendanceReportCsv_ליגת ווינר סל מחזור 20_ הפועל חיפה 🏠.csv',
    'AuthorizedAttendanceReportCsv_מחזור 22_ אליצור עירוני נתניה 🏠.csv',
    'AuthorizedAttendanceReportCsv_מחזור 24_ הפועל באר שבע - דימונה 🏠.csv',
    'AuthorizedAttendanceReportCsv_מחזור 26_ עירוני נס ציונה 🏠.csv',
    'AuthorizedAttendanceReportCsv_רבע גמר 1_ מכבי  עירוני רמת גן 🏠.csv',
    'AuthorizedAttendanceReportCsv_רבע גמר 3_ מכבי  עירוני רמת גן 🏠.csv',
    'AuthorizedAttendanceReportCsv_חצי גמר משחק 2_ הפועל תל אביב 🏠.csv',
    'AuthorizedAttendanceReportCsv_גמר ליגת ווינר משחק 2_ מכבי תל אביב 🏠.csv'
]

summary_list = []
raw_data_list = [] 

for file in game_files:
    game_df = pd.read_csv(file)
    game_df['Full Name'] = game_df['First name'] + " " + game_df['Last name']

    # Add Event Name
    event_name = file.split('Csv_')[1].replace('.csv', '')
    game_df['Event Name'] = event_name

    # Store unfiltered raw data
    raw_data_list.append(game_df.copy())

    # Filter only single ticket purchases
    game_df = game_df[(game_df['STRefID'].isna()) & (game_df['Type'] == 'Ticket')]

    # Use Voucher or CloseLink
    game_df['Name'] = np.where(
        game_df['Voucher name'].notna(),
        game_df['Voucher name'],
        game_df['CloseLink reservation name']
    )

    game_df['Type'] = np.where(
        game_df['Voucher name'].notna(),
        'Voucher',
        'CloseLink'
    )

    # Drop NA group names
    game_df = game_df.dropna(subset=['Name'])

    # Group by
    grouped = game_df.groupby(['Name']).size().reset_index(name='Count')
    grouped['Event Name'] = event_name

    grouped = grouped.merge(
        game_df[['Type', 'Name']].drop_duplicates(subset='Name'),
        on='Name',
        how='left'
    )

    summary_list.append(grouped)


working_games = pd.concat(summary_list, ignore_index=True)

# Define unwanted substrings
unwanted_substrings = ['חוץ', 'אורחת', 'רמת גן קישור לאוהדים', 'הפועל באר שבע דימונה', 'ניסיון קהילה']

# Filter out rows where Name contains any of the unwanted substrings
pattern = '|'.join(unwanted_substrings)
working_games = working_games[~working_games['Name'].str.contains(pattern, na=False)]


working_games = working_games[['Event Name', 'Name', 'Type', 'Count']].sort_values(by=['Event Name', 'Name'])
working_games.to_excel('working_games.xlsx')

all_raw_data = pd.concat(raw_data_list, ignore_index=True)

working_games

/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_31159/3250463268.py:23: DtypeWarning: Columns (16,21,31,35,38,40,44,45,64,65,70,71,75) have mixed types. Specify dtype option on import or set low_memory=False.
  game_df = pd.read_csv(file)


,Event Name,Name,Type,Count
0,Round 2_ Hapoel Tel Aviv 🚗,-50%t,Voucher,7
1,Round 2_ Hapoel Tel Aviv 🚗,-50t,Voucher,4
2,Round 2_ Hapoel Tel Aviv 🚗,00,Voucher,2
3,Round 2_ Hapoel Tel Aviv 🚗,lovehaphat,Voucher,9
4,Round 2_ Hapoel Tel Aviv 🚗,טסט קופון רב פעמי 1,Voucher,2
...,...,...,...,...
38,"🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת""א",HAPOELFAM2212,Voucher,1
39,"🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת""א",הפועל לב ירושלים,CloseLink,4
40,"🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת""א",טסט קופון רב פעמי 1,Voucher,2
41,"🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת""א",מאפס קרדיט,Voucher,1


### Same table with names and id of the people who used it:

In [14]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import re

def normalize_name(name):
    if pd.isna(name):
        return name
    name = str(name)
    name = name.replace("'", "")  # remove apostrophes
    name = name.replace('"', "")  # remove double quotes
    name = name.replace('אשה', 'אישה')  # fix missing י
    name = name.replace('בית ספר כרמים', 'בית הספר כרמים')  # fix missing י
    name = re.sub(r'\bה\s+', '', name)  # remove 'ה' if it's at beginning of a word
    name = re.sub(r'\s+', ' ', name)  # normalize multiple spaces
    name = name.strip()  # remove leading/trailing spaces
    return name


summary_list = []
raw_data_list = [] 

for file in game_files:
    game_df = pd.read_csv(file)
    game_df['Full Name'] = game_df['First name'] + " " + game_df['Last name']

    # Add Event Name
    event_name = file.split('Csv_')[1].replace('.csv', '')
    game_df['Event Name'] = event_name

    # Store unfiltered raw data
    raw_data_list.append(game_df.copy())

    # Filter only single ticket purchases
    game_df = game_df[(game_df['STRefID'].isna()) & (game_df['Type'] == 'Ticket')]

    # Use Voucher or CloseLink
    game_df['Name'] = np.where(
        game_df['Voucher name'].notna(),
        game_df['Voucher name'],
        game_df['CloseLink reservation name']
    )

    game_df['Type'] = np.where(
        game_df['Voucher name'].notna(),
        'Voucher',
        'CloseLink'
    )

    # Drop NA group names
    game_df = game_df.dropna(subset=['Name'])

    # Instead of grouping: just select columns
    selected = game_df[['Event Name', 'Name', 'Full Name', 'assign using  ID number', 'Type']].copy()

    selected['Count'] = 1  # each row = 1 ticket/person

    summary_list.append(selected)

working_games_with_ids = pd.concat(summary_list, ignore_index=True)

# Drop bad rows
working_games_with_ids = working_games_with_ids.dropna(subset=['assign using  ID number'])
working_games_with_ids = working_games_with_ids[
    working_games_with_ids['assign using  ID number'].astype(str).str.len() > 2
]
working_games_with_ids = working_games_with_ids.drop_duplicates()

# Normalize names
working_games_with_ids['Name_normalized'] = working_games_with_ids['Name'].apply(normalize_name)

# Fuzzy match similar names
unique_names = working_games_with_ids['Name_normalized'].unique()
name_mapping = {}

for name in unique_names:
    if name not in name_mapping:
        matches = process.extract(name, unique_names, scorer=fuzz.token_sort_ratio)
        for match_name, score in matches:
            if score > 90:  # adjust threshold if needed
                name_mapping[match_name] = name

# Apply mapping
working_games_with_ids['Name_normalized'] = working_games_with_ids['Name_normalized'].map(name_mapping).fillna(working_games_with_ids['Name_normalized'])

# Replace Name column
working_games_with_ids['Name'] = working_games_with_ids['Name_normalized']
working_games_with_ids = working_games_with_ids.drop(columns=['Name_normalized'])

# Define unwanted substrings
unwanted_substrings = ['חוץ', 'אורחת', 'רמת גן קישור לאוהדים', 'הפועל באר שבע דימונה', 'ניסיון קהילה']

# Filter out rows where Name contains any of the unwanted substrings
pattern = '|'.join(unwanted_substrings)
working_games_with_ids = working_games_with_ids[~working_games_with_ids['Name'].str.contains(pattern, na=False)]

# Final sorting
working_games_with_ids = working_games_with_ids.sort_values(by=['Event Name', 'Name'])

# Export to Excel
working_games_with_ids.to_excel('working_games_with_ids.xlsx', index=False)

working_games_with_ids

/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_31159/68811746.py:25: DtypeWarning: Columns (16,21,31,35,38,40,44,45,64,65,70,71,75) have mixed types. Specify dtype option on import or set low_memory=False.
  game_df = pd.read_csv(file)


,Event Name,Name,Full Name,assign using ID number,Type,Count
8,Round 2_ Hapoel Tel Aviv 🚗,-50%t,יקי קוזאהינוף,027124510,Voucher,1
11,Round 2_ Hapoel Tel Aviv 🚗,-50%t,גיל בש,GILBASH@GMAIL.COM,Voucher,1
13,Round 2_ Hapoel Tel Aviv 🚗,-50%t,אבי סמואלס,28327,Voucher,1
4,Round 2_ Hapoel Tel Aviv 🚗,-50t,יאיר מרינוב,11206,Voucher,1
2,Round 2_ Hapoel Tel Aviv 🚗,00,ELAD KASIR,217991017,Voucher,1
...,...,...,...,...,...,...
930,"🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת""א",צוות דובדבן,עילי ליכטנשטיין,326612280,CloseLink,1
931,"🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת""א",צוות דובדבן,איתי תורג׳מן,324187095,CloseLink,1
932,"🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת""א",צוות דובדבן,יונתן אליעז,216009829,CloseLink,1
949,"🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת""א",צוות דובדבן,נטלי אנגלמן,306930702,CloseLink,1


In [15]:
working_games_with_ids['Name'].unique()

array(['-50%t', '-50t', '00', 'lovehaphat', 'טסט קופון רב פעמי 1',
       'Complementary game tickets', 'אישה לאישה', 'בית הלוחם',
       'בית הספר בית הכרם', 'בית ספר תבל', 'הפועל ירושלים כדורעף',
       'כרטיס ב-50% הנחה', 'מועצה אזורית עין גדי', 'מיכאל.', 'מקיף גילה',
       '-100T', 'Credit', 'Resale', 'כרטיסי חבר גלריה', '-250', '100%T',
       'זיכוי', 'חצי גמר 2025', 'יאללה הדסה', 'כרטיס חבר B',
       'בית הספר כרמים', 'גדוד 53', 'הפועל אורן מודיעין', 'מכון סאמיט',
       'מרכז קהילתי גוננים', 'עמותת עדי', 'שמחה לילד', 'תיכון הימלפרב',
       'early0203', 'hjfamilyfriends', 'גימנסיה ירושלים', 'קרוס ריבר בנק',
       'תיכון מבשרת', '-10%T', 'NCSY', 'מכינת יונתן',
       'עובדי הדסה - קישור כללי', 'עמותת שבט הנובה', '-30%',
       'בית ילדים רמת רחל', 'בית ספר גאולים', 'כפר שאול',
       'כרטיסי ארגונים', 'מחלקת הנוער - הר אדר', 'מינהל קהילתי הר חומה',
       'הפועל לב ירושלים', 'כרטיסים להגרלה', '-40T', '-80T', '40 ALL',
       'קבוצת נוער רמת השרון', 'שוברי פסח 2025', 'שערי צדק

### All row data:

In [16]:
columns_to_keep = [
    'Event Id', 'assign using  ID number', 'First name', 'Last name', 'School', 'Gender', 'Age', 'Email', 'Id', 'Type', 'Event Name',
    'Additional phone number', 'Product', 'STRefID', 'OwnerSTUserId', 'OwnerSTName', 'Subtype Ticket number', 'Barcode', 'Card number',
    'User Id', 'Street', 'House number City', 'Country', 'Zip code', 'Stand', 'Area', 'Row', 'Number', 'Price area', 'Entrance code',
    'Entrance text', 'Transaction identifier', 'Transaction date', 'Delivery type', 'Price type', 'Price paid', 'Phone No.',
    'Transaction owner first name', 'Transaction owner last name', 'Transaction owner email', 'CloseLink reservation name',
    'CloseLink code', 'Voucher name', 'Voucher batch name', 'Discount price', 'Role', 'User group', 'Payment method', 'Ticket note',
    'Season tickets', 'Custom field 1 UserIdentityVerified', 'Attendance', 'Attendance date'
]

# Keep only columns that actually exist in the final DataFrame
existing_columns = [col for col in columns_to_keep if col in all_raw_data.columns]
all_raw_data = all_raw_data[existing_columns]

all_raw_data.to_excel('all_raw_data.xlsx')

all_raw_data

,Event Id,assign using ID number,First name,Last name,School,Gender,Age,Email,Id,Type,...,Voucher name,Voucher batch name,Discount price,Role,User group,Payment method,Ticket note,Season tickets,Attendance,Attendance date
0,1694,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,3045986,Ticket,...,NaN,NaN,No discount,Administrator,NaN,Cash,NaN,NaN,No,No data
1,1694,43552660,עינת,מידן-דוד,NaN,Female,42.0,einatth@gmail.com,3046265,Ticket,...,NaN,NaN,No discount,Fan,NaN,Other,NaN,NaN,No,No data
2,1694,236546305,נועם,אהרוני,NaN,Unknown,1.0,NaN,3050463,Ticket,...,NaN,NaN,No discount,Administrator,NaN,Pelecard_Credit Card,NaN,NaN,No,No data
3,1694,43467760,עדו,מידן דוד,NaN,Male,43.0,idomedan@gmail.com,3053065,Ticket,...,NaN,NaN,No discount,Administrator,"פרימיום, חיילים בסדיר, מילואים, Hapoel Test gr...",Cash,NaN,NaN,No,No data
4,1694,215015306,עילי,שליו,NaN,Unknown,20.0,NaN,3053295,Ticket,...,NaN,NaN,No discount,Fan,NaN,Pelecard_Credit Card,NaN,NaN,Yes,2024-10-13 20:05:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90263,4172,339027559,זיו,קציר,NaN,Unknown,11.0,NaN,899550,SeasonTicket,...,NaN,NaN,No discount,Administrator,NaN,External Payment Subscriptions,NaN,NaN,No,No data
90264,4172,222664401,מיטל,הערצקה,NaN,Unknown,10.0,NaN,899556,SeasonTicket,...,NaN,NaN,No discount,Administrator,NaN,Pelecard_Credit Card,NaN,NaN,No,No data
90265,4172,332578384,איילה,הערצקה,NaN,Female,15.0,isaacherzka8@gmail.com,899557,SeasonTicket,...,NaN,NaN,No discount,Administrator,NaN,Pelecard_Credit Card,NaN,NaN,No,No data
90266,4172,25304320,ערן,ויס,NaN,Male,51.0,eranwe36@gmail.com,899563,SeasonTicket,...,NaN,NaN,No discount,Administrator,NaN,Pelecard_Credit Card,NaN,NaN,No,No data
